<a href="https://www.kaggle.com/code/yaaangzhou/autogluon-gbm-cat-xgb-and-more?scriptVersionId=135619562" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [2]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 750.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 50.0 MB/s e

In [3]:
import pandas as pd
import numpy as np
import autogluon 

In [4]:
train_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
sample_submission_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')

In [5]:
train_df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [6]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(train_df)

label = 'Class'

predictor = TabularPredictor(label=label).fit(train_data,
                                              time_limit=3600,
                                              verbosity=3,)

predictor.leaderboard()

No path specified. Models will be saved in: "AutogluonModels/ag-20230703_162958/"
============ fit kwarg info ============
User Specified kwargs:
{'verbosity': 3}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': False,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'included_model_types': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
Saving AutogluonModels/ag-20230703_162958/learner.pkl
Saving AutogluonModels/ag-20230703_162958/predictor.pkl
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will 

[50]	valid_set's binary_error: 0.0887097
[100]	valid_set's binary_error: 0.0967742
[150]	valid_set's binary_error: 0.0887097
[200]	valid_set's binary_error: 0.0725806
[250]	valid_set's binary_error: 0.0725806
[300]	valid_set's binary_error: 0.0725806
[350]	valid_set's binary_error: 0.0645161
[400]	valid_set's binary_error: 0.0645161
[450]	valid_set's binary_error: 0.0645161
[500]	valid_set's binary_error: 0.0725806
[550]	valid_set's binary_error: 0.0645161


Saving AutogluonModels/ag-20230703_162958/models/LightGBMXT/model.pkl
Saving AutogluonModels/ag-20230703_162958/utils/attr/LightGBMXT/y_pred_proba_val.pkl
	0.9355	 = Validation score   (accuracy)
	2.77s	 = Training   runtime
	0.01s	 = Validation runtime
Saving AutogluonModels/ag-20230703_162958/models/trainer.pkl
Fitting model: LightGBM ... Training model for up to 3596.13s of the 3596.11s of remaining time.
	Fitting LightGBM with 'num_gpus': 0, 'num_cpus': 1
	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05}


[600]	valid_set's binary_error: 0.0645161
[50]	valid_set's binary_error: 0.0887097
[100]	valid_set's binary_error: 0.0806452
[150]	valid_set's binary_error: 0.0806452
[200]	valid_set's binary_error: 0.0806452
[250]	valid_set's binary_error: 0.0887097
[300]	valid_set's binary_error: 0.0887097


Saving AutogluonModels/ag-20230703_162958/models/LightGBM/model.pkl
Saving AutogluonModels/ag-20230703_162958/utils/attr/LightGBM/y_pred_proba_val.pkl
	0.9355	 = Validation score   (accuracy)
	0.84s	 = Training   runtime
	0.0s	 = Validation runtime
Saving AutogluonModels/ag-20230703_162958/models/trainer.pkl
Fitting model: RandomForestGini ... Training model for up to 3595.27s of the 3595.26s of remaining time.
	Fitting RandomForestGini with 'num_gpus': 0, 'num_cpus': 2


...

Saving AutogluonModels/ag-20230703_162958/models/XGBoost/model.pkl
Saving AutogluonModels/ag-20230703_162958/utils/attr/XGBoost/y_pred_proba_val.pkl
	0.9113	 = Validation score   (accuracy)
	0.62s	 = Training   runtime
	0.01s	 = Validation runtime
Saving AutogluonModels/ag-20230703_162958/models/trainer.pkl
Fitting model: NeuralNetTorch ... Training model for up to 3582.2s of the 3582.18s of remaining time.
	Fitting NeuralNetTorch with 'num_gpus': 0, 'num_cpus': 1
Tabular Neural Network treats features as the following types:
{
    "continuous": [
        "AZ",
        "BN",
        "CU",
        "CW ",
        "DH",
        "DN",
        "EL",
        "GH",
        "GL"
    ],
    "skewed": [
        "AB",
        "AF",
        "AH",
        "AM",
        "AR",
        "AX",
        "AY",
        "BC",
        "BD ",
        "BP",
        "BQ",
        "BR",
        "BZ",
        "CB",
        "CC",
        "CD ",
        "CF",
        "CH",
        "CL",
        "CR",
        "CS",
 

[50]	valid_set's binary_error: 0.129032
[100]	valid_set's binary_error: 0.0887097
[150]	valid_set's binary_error: 0.104839
[200]	valid_set's binary_error: 0.104839
[250]	valid_set's binary_error: 0.0967742
[300]	valid_set's binary_error: 0.0887097
[350]	valid_set's binary_error: 0.0887097
[400]	valid_set's binary_error: 0.0887097
[450]	valid_set's binary_error: 0.0967742
[500]	valid_set's binary_error: 0.104839
[550]	valid_set's binary_error: 0.0967742


Saving AutogluonModels/ag-20230703_162958/models/LightGBMLarge/model.pkl
Saving AutogluonModels/ag-20230703_162958/utils/attr/LightGBMLarge/y_pred_proba_val.pkl
	0.9194	 = Validation score   (accuracy)
	2.17s	 = Training   runtime
	0.01s	 = Validation runtime
Saving AutogluonModels/ag-20230703_162958/models/trainer.pkl
Loading: AutogluonModels/ag-20230703_162958/utils/attr/RandomForestEntr/y_pred_proba_val.pkl
Loading: AutogluonModels/ag-20230703_162958/utils/attr/CatBoost/y_pred_proba_val.pkl
Loading: AutogluonModels/ag-20230703_162958/utils/attr/KNeighborsDist/y_pred_proba_val.pkl
Loading: AutogluonModels/ag-20230703_162958/utils/attr/ExtraTreesEntr/y_pred_proba_val.pkl
Loading: AutogluonModels/ag-20230703_162958/utils/attr/XGBoost/y_pred_proba_val.pkl
Loading: AutogluonModels/ag-20230703_162958/utils/attr/NeuralNetTorch/y_pred_proba_val.pkl
Loading: AutogluonModels/ag-20230703_162958/utils/attr/LightGBMXT/y_pred_proba_val.pkl
Loading: AutogluonModels/ag-20230703_162958/utils/attr/Ex

                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.951613       0.020734  5.812264                0.001104           0.822899            2       True         14
1              LightGBM   0.935484       0.003340  0.838881                0.003340           0.838881            1       True          4
2            LightGBMXT   0.935484       0.006163  2.767770                0.006163           2.767770            1       True          3
3              CatBoost   0.927419       0.005181  6.286528                0.005181           6.286528            1       True          7
4        NeuralNetTorch   0.927419       0.043284  2.957655                0.043284           2.957655            1       True         12
5         LightGBMLarge   0.919355       0.007902  2.168899                0.007902           2.168899            1       True         13
6               XGBoost   0.911290

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.951613,0.020734,5.812264,0.001104,0.822899,2,True,14
1,LightGBM,0.935484,0.003340,0.838881,0.003340,0.838881,1,True,4
2,LightGBMXT,0.935484,0.006163,2.767770,0.006163,2.767770,1,True,3
3,CatBoost,0.927419,0.005181,6.286528,0.005181,6.286528,1,True,7
4,NeuralNetTorch,0.927419,0.043284,2.957655,0.043284,2.957655,1,True,12
5,LightGBMLarge,0.919355,0.007902,2.168899,0.007902,2.168899,1,True,13
6,XGBoost,0.911290,0.005716,0.623925,0.005716,0.623925,1,True,11
7,RandomForestEntr,0.911290,0.077207,0.864542,0.077207,0.864542,1,True,6
8,NeuralNetFastAI,0.903226,0.013467,2.221595,0.013467,2.221595,1,True,10
9,RandomForestGini,0.895161,0.077552,0.861944,0.077552,0.861944,1,True,5


In [7]:
pred = predictor.predict_proba(test_df)

Loading: AutogluonModels/ag-20230703_162958/models/LightGBMXT/model.pkl
Loading: AutogluonModels/ag-20230703_162958/models/NeuralNetFastAI/model.pkl
Loading: AutogluonModels/ag-20230703_162958/models/NeuralNetFastAI/model-internals.pkl
Loading: AutogluonModels/ag-20230703_162958/models/WeightedEnsemble_L2/model.pkl


In [8]:
pred

,0,1
0,0.808243,0.191757
1,0.808243,0.191757
2,0.808243,0.191757
3,0.808243,0.191757
4,0.808243,0.191757


In [9]:
sample_submission_df['Id'] = test_df.reset_index()['Id']
sample_submission_df['class_0'] = pred[0]
sample_submission_df['class_1']  = pred[1]

sample_submission_df

,Id,class_0,class_1
0,00eed32682bb,0.808243,0.191757
1,010ebe33f668,0.808243,0.191757
2,02fa521e1838,0.808243,0.191757
3,040e15f562a2,0.808243,0.191757
4,046e85c7cc7f,0.808243,0.191757


In [10]:
sample_submission_df.set_index('Id').to_csv('submission.csv')